In [1]:
import json, requests
import pandas as pd
import numpy as np
import io
import warnings
warnings.filterwarnings('ignore')

from joblib import dump, load

In [2]:
import matplotlib as mlab
import matplotlib.pyplot as plt
import seaborn as sns 
import time
import math

# import statsmodels.api as sm    
# import scipy.stats as stats 
# from pylab import rcParams

import sys
from functools import reduce
import itertools
import random

import datetime
from datetime import date, timedelta
from pandas.tseries.offsets import Day, MonthBegin, MonthEnd

# Libraries for stock data
import yfinance as yf

# from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error
# from sklearn.metrics import mean_squared_error
# from sklearn.metrics import mean_squared_error as MSE
# from sklearn.metrics import accuracy_score, mean_absolute_percentage_error, mean_absolute_error
# from sklearn.metrics import confusion_matrix, classification_report

# from sklearn.decomposition import PCA
# from sklearn.manifold import TSNE
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import train_test_split


import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models, layers
# from tensorflow.keras.layers import Conv1D,BatchNormalization, Dropout, Flatten, Input, Dense
# from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
# from tensorflow.keras.models import Sequential

2023-01-18 17:04:39.025869: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-18 17:04:44.833798: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-01-18 17:04:44.834258: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-01-18 17:04:44.834267: W 

In [3]:
import random
from random import sample
from transform_forecaster import stock_data_pull, fill_blanks, \
     change_shape, dataset, window_gen; 

Procedure for testing:
1. test model response with randomly generated data.
2. test model with data sourced from y finance (just a hand full of symbols)
3. include pre-process steps similar to those in direction model
4. Test model within .py file

In [4]:
recon_model = keras.models.load_model('/home/cdsw/saved_models/model_01_17_23')

2023-01-18 17:04:52.031079: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-01-18 17:04:52.031108: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-01-18 17:04:52.031129: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (pjo408xvxdusfnol): /proc/driver/nvidia/version does not exist
2023-01-18 17:04:52.031352: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler fl

**Random input data**

In [5]:
randomlist = []
for i in range(0,60):
  n = random.randint(95,130)
  randomlist.append(n)
# converting list to array
arr = np.array(randomlist)

test = arr.reshape(1,arr.shape[0],1)

pred = recon_model.predict(test)

1/1 [==============================] - 1s 1s/step


In [6]:
pred

array([[108.01795 , 108.65435 , 108.82354 , 109.56932 , 109.69296 ,
        110.41506 , 109.636856, 109.658806, 109.29382 , 109.59643 ,
        110.33948 , 110.190025]], dtype=float32)

In [7]:
#!ls /home/cdsw/saved_models

**Load yfinance data**

In [8]:
table=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
tckr = table[0]
# convert to list
basket = tckr.Symbol.unique().tolist()

In [9]:
smp_basket = sample(basket,10)

In [10]:
# Pull stock data for given ticker symvols
stock_total = stock_data_pull(smp_basket,False);

Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'Name'], dtype='object')


In [19]:
stock_total.Datetime.drop_duplicates().sort_values()

0      2022-12-19 09:30:00-05:00
1      2022-12-19 09:35:00-05:00
2      2022-12-19 09:40:00-05:00
3      2022-12-19 09:45:00-05:00
4      2022-12-19 09:50:00-05:00
                  ...           
1508   2023-01-18 11:45:00-05:00
1509   2023-01-18 11:50:00-05:00
1510   2023-01-18 11:55:00-05:00
1511   2023-01-18 12:00:00-05:00
3025   2023-01-18 12:05:00-05:00
Name: Datetime, Length: 1514, dtype: datetime64[ns, America/New_York]

In [21]:
window = 60
date_range = stock_total.Datetime.drop_duplicates()
# added line below to clip off the lasest data
date_range = date_range.sort_values().tail(window)
date_range = pd.DataFrame(date_range)
stock_ready = fill_blanks(stock_total,'Name',date_range,'Datetime');

In [79]:
# date_range.sort_values('Datetime').tail(10)
# look at resultant dataframe
stock_ready.head()

stock_ready.dtypes

Datetime    datetime64[ns, America/New_York]
Close                                float64
Volume                               float64
Name                                  object
dtype: object

In [22]:
# check that all stocks have equal sizes
sizes = stock_ready.groupby('Name').size().unique()
sizes

array([60])

#### Generate predictions for actual stocks

In [26]:
stock_ready.Name.unique().shape

(10,)

In [35]:
symbol_list = []
for i, tckr in enumerate(stock_ready.Name.unique()):
    symbol_list.append(tckr)
    temp_input = stock_ready[stock_ready.Name == tckr].copy().Close.values
    if i == 0:
        input_array = temp_input
    else:
        input_array = np.append(input_array,temp_input)

In [40]:
input_array = input_array.reshape(i+1,window,1)

In [41]:
pred = recon_model.predict(input_array)

1/1 [==============================] - 1s 763ms/step


In [42]:
pred.shape

(10, 12)

In [48]:
i+1

10

In [49]:
#pred.tolist()
pred_time_list = [stock_ready.Datetime.max()]*(i+1)

In [53]:
tst_output = {'symbol':symbol_list,'pred_time':pred_time_list,'prediction':pred.tolist()}

Test .py file with forecating model

In [58]:
from fct_model_serve import forecast_price


Not running in a model replica, so using a local development
version of the model metrics service. Please use the following
CRN's to consume metrics:
   model_crn: "crn:cdp:ml:::workspace:dev/model" (cdsw.dev_model_crn)
   model_build_crn: "crn:cdp:ml:::workspace:dev/model-build" (cdsw.dev_model_build_crn)
   model_deployment_crn: "crn:cdp:ml:::workspace:dev/model-deployment" (cdsw.dev_model_deployment_crn)



#### Test deployed Model

In [61]:
# use this as input data
cols = ['symbol','symbol_time','close', 'volume']
stock_total.columns = ['symbol_time','close','volume','symbol']
stock_total.head()

,symbol_time,close,volume,symbol
0,2022-12-19 09:30:00-05:00,112.809998,16777,SBNY
1,2022-12-19 09:35:00-05:00,113.769997,2069,SBNY
2,2022-12-19 09:40:00-05:00,113.410004,9969,SBNY
3,2022-12-19 09:45:00-05:00,112.949997,8870,SBNY
4,2022-12-19 09:50:00-05:00,112.730003,4686,SBNY


In [59]:
model_url = 'https://modelservice.ml-92202c44-d46.loki-cdp.a465-9q4k.cloudera.site/model'
accessKey = 'm45a083on9okf24psw58d0v8j6ht6w3u'

In [78]:
stock_total.dtypes

symbol_time    datetime64[ns, America/New_York]
close                                   float64
volume                                    int64
symbol                                   object
dtype: object

In [62]:
cols = stock_total.columns

In [73]:
# load test data
df = pd.read_csv('test_model_input.csv')

In [77]:
df.groupby('Datetime').size()
#df.head()

Datetime
2023-01-09 13:35:00    10
dtype: int64

In [83]:
stock_total.symbol_time = stock_total.symbol_time.astype("str") 

In [84]:
stock_total.dtypes

symbol_time     object
close          float64
volume           int64
symbol          object
dtype: object

In [85]:
# build input for api into dictionary w/ values as a list
input_data_mult = dict()
for col in cols:
    input_data_mult[col] = stock_total[col].to_list()    
    
# build embedded dictionary step 1    
request_dict = {"request":input_data_mult}

# access key will be end point specific
BackDict = {"accessKey":accessKey}
BackDict.update(request_dict)
request_dict=BackDict

In [68]:
type(request_dict)

dict

In [88]:
# json.dumps(request_dict)

In [89]:
r = requests.post(model_url, data=json.dumps(request_dict),headers={'Content-Type': 'application/json'})

In [90]:
r.json()

{'success': True,
 'response': {'model_deployment_crn': 'crn:cdp:ml:us-west-1:558bc1d2-8867-4357-8524-311d51259233:workspace:17467d56-4523-429e-ac77-5f98c4894f90/fcbefc46-4ff3-46de-beb2-19f43f6b4df4',
  'prediction': {'pred_time': ['2023-01-18 12:05:00-05:00',
    '2023-01-18 12:05:00-05:00',
    '2023-01-18 12:05:00-05:00',
    '2023-01-18 12:05:00-05:00',
    '2023-01-18 12:05:00-05:00',
    '2023-01-18 12:05:00-05:00',
    '2023-01-18 12:05:00-05:00',
    '2023-01-18 12:05:00-05:00',
    '2023-01-18 12:05:00-05:00',
    '2023-01-18 12:05:00-05:00'],
   'prediction': [[119.3626708984375,
     119.26956176757812,
     119.2031478881836,
     119.20362854003906,
     119.16153717041016,
     119.1254653930664,
     119.1527328491211,
     119.0446548461914,
     119.14306640625,
     119.15259552001953,
     119.17501068115234,
     119.20993041992188],
    [324.0056457519531,
     323.9454345703125,
     323.8894348144531,
     323.925537109375,
     323.8377990722656,
     323.865478

In [ ]:
# def mkt_movement(args):
#     filtArgs = {key:[args[key]] for key in features}
#     data = pd.DataFrame.from_dict(filtArgs)
    
#     pipe_rf = load("saved_models/pipe_rf.joblib")

#     prediction = pipe_rf.predict(data)
#     print(prediction)
        
#     return prediction